# Project Part 1: Data Preparation 

For the twitter sentiment analysis, I have used the dataset of tweets collected from the sports event of the 2018 UEFA Champions League final match at 26 May 2018, Real Madrid vs Liverpool. 

The data was collected over four weeks since from 21 May and end at 16 June 2018 at pre-, mid- and post-event.   

The description of the dataset can be found as follows. 

- created_at: UTC time when this Tweet was created
- name: The name of the user who posted this tweet. See User data diction attributes
- screen_name: The screen name of the user who posted this tweet. It can be either same or different from user's name 
- id: The id of the user who posted this tweet
- friends: The number of users this account is following
- followers: The number of followers this account currently has
- listed: The number of public lists that this user has posted
- text: Contain a truncated version of the Tweet text, followed by an ellipsis character, a space, and a shortened self-permalink URL. The total length of this text value shall not exceed 140 characters
- full_text: Contains the full, untruncated Tweet text
- ideo_score: Created as empty obejects for the further analysis of ideology preference. 

# First Look at the data

In [2]:
import json
import twitter
from twitter import *
import os

import pandas as pd
import numpy as np
os.chdir('')

In [52]:
data = []
with open("madrid_pre.json") as f:
    # use 'marid_post_week3 as a test sample'
    for line in f:
        try:
            tweet = json.loads(line)
            if tweet['text'].find('RT @')!=-1:
            # drop Retweeted tweet
                continue
            data.append(tweet)
        except:
            continue

with open('nmadrid_pre.json','w') as fp:
    json.dump(data,fp)

tweets_data_path=('nmadrid_pre.json')
tweets_data = []
tweets_file = open(tweets_data_path,'r')
tweets_data = json.load(tweets_file)
type(tweets_data)

# convert the json file to dataframe
df = pd.DataFrame(tweets_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7101 entries, 0 to 7100
Data columns (total 35 columns):
contributors                 0 non-null object
coordinates                  3 non-null object
created_at                   7101 non-null object
display_text_range           5915 non-null object
entities                     7101 non-null object
extended_entities            521 non-null object
extended_tweet               2159 non-null object
favorite_count               7101 non-null int64
favorited                    7101 non-null bool
filter_level                 7101 non-null object
geo                          3 non-null object
id                           7101 non-null int64
id_str                       7101 non-null object
in_reply_to_screen_name      4554 non-null object
in_reply_to_status_id        4161 non-null float64
in_reply_to_status_id_str    4161 non-null object
in_reply_to_user_id          4554 non-null float64
in_reply_to_user_id_str      4554 non-null object
is_qu

I first started by dropping the columns that I don't need for the specific purpose of sentiment analysis. I only need certain elements i.e., user's profile, text, date, thus extract those elements and save to a new dataframe with 4051 length. 

In [53]:
col_to_keep = ['created_at','user','text','extended_tweet']
df = df[col_to_keep]

add_followers = []
add_friends = []
add_listed = []
add_id = []
add_name = []
add_screename = []

# extract elements from dataframe series 
for index, row in df.iterrows():
    fo = (row['user']['followers_count'])
    fr = (row['user']['friends_count'])
    li = (row['user']['listed_count'])
    id = (row['user']['id'])
    na = (row['user']['name'])
    sc = (row['user']['screen_name'])
    
    add_followers.append(fo)
    add_friends.append(fr)
    add_listed.append(li)
    add_id.append(id)
    add_name.append(na)
    add_screename.append(sc)

    
df['followers'] = add_followers
df['friends'] = add_friends
df['listed'] = add_listed
df['id'] = add_id
df['name'] = add_name
df['screen_name'] = add_screename
# 'ideology_score' column generated for further analysis 
df['ideo_score'] = ''

ncol_to_keep = ['created_at','name','screen_name','id','friends','followers','listed','text','extended_tweet','ideo_score']
df = df[ncol_to_keep]
# the elementes needed for the further analysis are saved in current dataframe
df.head()

# Store the data to a csv file
pd.to_csv('madrid_pre.csv', encoding='utf-8')

,created_at,name,screen_name,id,friends,followers,listed,text,extended_tweet,ideo_score
0,Mon May 21 00:48:53 +0000 2018,mwa150,MengjieW,1630534146,79,16,2,@realmadridfra forwarding this sat!,NaN,
1,Mon May 21 01:16:44 +0000 2018,Yago Amaral,yagoorocha11,3253070183,322,66,0,@luka7doncic I want a Doncic Real Madrid jerse...,NaN,
2,Mon May 21 01:29:22 +0000 2018,Patricia,Patricia_gb11,917424322943307776,539,263,0,happy night my life i love you always my idol ...,"{u'display_text_range': [0, 146], u'entities':...",
3,Mon May 21 01:44:58 +0000 2018,Binyamin Alsagoff,benalsagoff676,325975611,211,221,3,Countdown to @ChampionsLeague 2018 Final in Ky...,NaN,
4,Mon May 21 01:56:23 +0000 2018,NİCOLO MELLİH,vamoslaramos,925375657839878144,299,1006,1,@Okechukwu1907 @realmadrid Lovee,NaN,


# User Filters

I limit the sample to only active users, which is defined as those who a) have at least 25 followers, b) follow 50 or more other accounts, c) tweet in English, and d) have sent more than 3 tweets.  

For example, the 1st user shown in the dataframe below was created for a testing purpose, which should be removed by filtering.  

In [54]:
df.describe()
# description of friends/follower/listed in the previous dataframe

df = df[df['friends'] >= 50]
df = df[df['followers'] >= 25]
df = df[df['listed'] >=3]

,id,friends,followers,listed
count,7.101000e+03,7101.000000,7101.000000,7101.000000
mean,-6.329437e+14,925.918603,46901.008309,77.500915
std,4.444126e+17,6754.944716,778821.256344,850.950874
min,6.542030e+05,0.000000,0.000000,0.000000
25%,3.881136e+08,113.000000,36.000000,0.000000
50%,2.634617e+09,319.000000,204.000000,1.000000
75%,8.849007e+17,749.000000,742.000000,9.000000
max,1.000145e+18,383675.000000,21478381.000000,22337.000000


In [56]:
df.describe()
# description of friends/follower/listed in the present dataframe that contains all 'active' users

,id,friends,followers,listed
count,2.796000e+03,2796.000000,2796.000000,2796.000000
mean,4.806150e+14,1770.631617,118087.332976,194.208155
std,2.555179e+17,10684.502368,1237683.923388,1346.987286
min,6.542030e+05,50.000000,27.000000,3.000000
25%,1.941524e+08,332.000000,373.750000,5.000000
50%,5.094155e+08,655.500000,902.000000,13.000000
75%,2.399922e+09,1310.750000,2997.000000,42.000000
max,9.959838e+17,383675.000000,21478381.000000,22337.000000


# Dataframe Merge

In [34]:
madrid_df1 = pd.read_csv('nmadrid_pre.csv')
madrid_df2 = pd.read_csv('nmadrid_mid.csv')
madrid_df3 = pd.read_csv('nmadrid_post_week1.csv')
madrid_df4 = pd.read_csv('nmadrid_post_week2.csv')
madrid_df5 = pd.read_csv('nmadrid_post_week3.csv')

madrid_df = pd.concat([madrid_df1, madird_df2, madrid_df3, madrid_df4, madrid_df5])
# save as a new csv
madrid_df.to_csv('madrid_df.csv')

print(madrid_df.head())
print(madrid_df.tail())

                       created_at                name      screen_name  \
0  Mon May 21 01:44:58 +0000 2018   Binyamin Alsagoff   benalsagoff676   
1  Mon May 21 04:01:00 +0000 2018              LaLiga           LaLiga   
2  Mon May 21 04:08:40 +0000 2018  Vida Social Bogotá   SocialEnBogota   
3  Mon May 21 04:38:32 +0000 2018                 TD.         iamthetd   
4  Mon May 21 04:52:52 +0000 2018  bigtime or bigmike  Michaelgreatho2   

           id  friends  followers  listed  \
0   325975611      211        221       3   
1   423384542      828    3632180    5536   
2  2611748882    14598      34695     290   
3  1066034574      557         94       6   
4  2440701187     1918        742       4   

                                                text  \
0  Countdown to @ChampionsLeague 2018 Final in Ky...   
1  ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @r...   
2  ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @r...   
3  Well, @realmadrid ended this seasons @LaLiga c...   
4  @

In [40]:
liverp_df1 = pd.read_csv('nliverpool_pre.csv')
liverp_df2 = pd.read_csv('nliverpool_mid.csv')
liverp_df3 = pd.read_csv('nliverpool_post_week1.csv')
liverp_df4 = pd.read_csv('nliverpool_post_week2.csv')
liverp_df5 = pd.read_csv('nliverpool_post_week3.csv')

liverp_df = pd.concat([liverp_df1, liverp_df2, liverp_df3, liverp_df4, liverp_df5])
# save as a new csv
liverp_df.to_csv('liverp_df.csv')

print(liverp_df.head())
print(liverp_df.tail())

                       created_at                  name   screen_name  \
0  Mon May 21 01:15:18 +0000 2018        Rami Elhussein       ramizak   
1  Mon May 21 01:16:48 +0000 2018  The English American      an_esq10   
2  Mon May 21 01:23:34 +0000 2018           Julianto HG  sersankancil   
3  Mon May 21 01:23:53 +0000 2018      nathan middleton   nathanjm000   
4  Mon May 21 01:28:09 +0000 2018  Fantasy Sport Source  superfan1623   

             id  friends  followers  listed  \
0  2.741735e+07      929        388      19   
1  7.603282e+08      724        349       8   
2  1.723466e+08      178        263       6   
3  8.190000e+17     4971       2840      10   
4  3.461252e+08     1293       1051      13   

                                                text  \
0  @nicktanner799 @ayushmalla77 @LFC Sad news but...   
1  @EL_PISTOLERO_07 @LFC Why would you want De Vr...   
2  Follow @LFC @LiverpudlianLFC @OfficialLFC_ID #...   
3  @EL_PISTOLERO_07 @LFC He has already signed FO...  

The two new dataframes are generated which contains all timeslots. Length with 35010 rows in madrid_df and 145509 rows in liverp_df (verify as below). 

In [47]:
print(np.sum([len(madrid_df1), len(madrid_df2),len(madrid_df3),len(madrid_df4),len(madrid_df5)]))
print(np.sum([len(liverp_df1), len(liverp_df2),len(liverp_df3),len(liverp_df4),len(liverp_df5)]))
print(len(madrid_df))
print(len(liverp_df))

35010
145509
35010
145509


# Project Part 2: Text Cleaning 

One of the 1st steps in working with twitter text is to pre-process it. The most of text data is highly unstructured and noisy - to achieve better insights or to build better algorithms, it is necessary to play with clean data. 

The possible noise elements that should be removed as follows: 


- Decoding data: transforming information from complex symbols to simple and easier to understand characters (i.e., UTF-8 encoding is widely accepted and is recommended to use)
- HTML characters: it looks like the data obtained from web usually contains a lot of html entities such as &amp in the text field, which should be removed
- URLs: URLs and hyperlinks in text data like comments, reviews, and tweets should be removed
- Emoticons: textual data can contain facial expression using characters. This information doesn't add value to build sentiment analysis model
- @mention: same with emoticons, even though it carries some information, for sentiment analysis purpose, this can be ignored
- Hash-tags: leave the text intact and just remove the '#'
- Stop-words: the commonly occurring words such as 'a', 'the', 'it', 'to' should be removed. One can either create a long list of stop-words or one can use predefined language specific libraries
- Split: check if there's any contracted or negated term i.e., 'isn't', and replace it with its original form

Examples can be found as follows. 

## Text Cleaning : Removal of Stop-words

In [5]:
pd.set_option('display.max_colwidth', -1)

In [6]:
df = pd.read_csv('madrid_df.csv')
test_text = df['text'][:1]

from nltk.corpus import stopwords
import string
replace_set = stopwords.words('english')
no_stopwords = test_text.str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set))

print test_text
print no_stopwords

0    Countdown to @ChampionsLeague 2018 Final in Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy
Name: text, dtype: object
0    Countdown @ChampionsLeague 2018 Final Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy
Name: text, dtype: object


## Text Cleaning : Decoding and Escaping HTML 

In [35]:
test_text2 = df['text'][83]
print(test_text2)
test_text2_decoding = test_text2.decode("utf8").encode('ascii','ignore')

import HTMLParser
html_parser = HTMLParser.HTMLParser()
no_html = html_parser.unescape(test_text2_decoding)
print(no_html)

Have a read of the latest Q&amp;A from @_Chris_Hurst_ on @LiverpoolFCHQ_ 🔴🇺🇦 https://t.co/VlAFdgPy4s
Have a read of the latest Q&A from @_Chris_Hurst_ on @LiverpoolFCHQ_  https://t.co/VlAFdgPy4s


## Text Cleaning : URL links

In [36]:
import re

no_urls = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+','',no_html)
print(no_urls)

Have a read of the latest Q&A from @_Chris_Hurst_ on @LiverpoolFCHQ_  


## Text Cleaning : @mention

In [37]:
no_atmention = re.sub(r'@[\w_]+','', no_urls)
print(no_atmention)

Have a read of the latest Q&A from  on   


## Text Cleaning : Emoticons

In [38]:
test_text3 = df['text'][11]
print(test_text3)

re.sub(r'[:=;] [oO\-]?[D\)\]\(\]/\\OpP]', '', test_text3)

@socraticjuan How could @realmadrid do that atrocity to @IkerCasillas i feel sorry for him 😠😠😢😢


'@socraticjuan How could @realmadrid do that atrocity to @IkerCasillas i feel sorry for him \xf0\x9f\x98\xa0\xf0\x9f\x98\xa0\xf0\x9f\x98\xa2\xf0\x9f\x98\xa2'

## Defining Text Cleaning Function

Now definding a function for text cleaning and apply to the whole dataset. Tokenization, stemming/lemmatization will be dealt with later stage when creating matrix with either count vectorizer or Tfidf vectorizer.

In [7]:
df_madrid = pd.read_csv('madrid_df.csv')
demo = df_madrid['text'][:10] # demo testing with 10 texts 
demo

0    Countdown to @ChampionsLeague 2018 Final in Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy                      
1    ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @realmadrid! 💥 https://t.co/YHfrJ3E6B3                                                         
2    ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @realmadrid ! 💥 https://t.co/NTLogiPwl1                                                        
3    Well, @realmadrid ended this seasons @LaLiga campaign the same way they started it, shitty. I just hope that Zidane… https://t.co/zGxANmIWu7
4    @BleacherReport @Cristiano @realmadrid Yall late https://t.co/psBUG5iAdB                                                                    
5    @LaLiga @Cristiano @realmadrid Yall late https://t.co/Nl1HSiA7fK                                                                            
6    "@LaLiga: ¡@Cristiano llega a los 4⃣5⃣0⃣ goles con el @realmadrid! 💥 https://t.co/0ScNLgSKNM"                            

In [8]:
replace_set = stopwords.words('english') 
demo = demo.str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set)) # texts with no stop-words
demo

C:\Users\yaffy\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


0    Countdown @ChampionsLeague 2018 Final Kyiv, Russia. @LFC vs @realmadrid #YNWA #6DaysToGo https://t.co/Ixx0jnPdNy
1    ¡@Cristiano llega los 4⃣5⃣0⃣ goles con el @realmadrid! 💥 https://t.co/YHfrJ3E6B3                               
2    ¡@Cristiano llega los 4⃣5⃣0⃣ goles con el @realmadrid ! 💥 https://t.co/NTLogiPwl1                              
3    Well, @realmadrid ended seasons @LaLiga campaign way started it, shitty. I hope Zidane… https://t.co/zGxANmIWu7 
4    @BleacherReport @Cristiano @realmadrid Yall late https://t.co/psBUG5iAdB                                        
5    @LaLiga @Cristiano @realmadrid Yall late https://t.co/Nl1HSiA7fK                                                
6    "@LaLiga: ¡@Cristiano llega los 4⃣5⃣0⃣ goles con el @realmadrid! 💥 https://t.co/0ScNLgSKNM"                    
7    @BleacherReport @Cristiano @realmadrid Score 450 goal may 6 yall late                                           
8    I really hope Pérez brings @neymarjr Bernabéu summer. 

Splitting terms with contraction and negation 

In [9]:
split_dic = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
             "can't've": "cannot have", "'cause": "because", "could've": "could have", 
             "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
             "doesn't": "does not", "don't": "do not", "hadn't": "had not",  
             "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
             "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
             "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
             "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
             "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
             "I'll've": "I will have","I'm": "I am", "I've": "I have", 
             "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
             "i'll've": "i will have","i'm": "i am", "i've": "i have", 
             "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
             "it'll": "it will", "it'll've": "it will have","it's": "it is", 
             "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
             "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
             "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
             "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
             "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
             "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
             "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
             "she's": "she is", "should've": "should have", "shouldn't": "should not", 
             "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
             "this's": "this is",
             "that'd": "that would", "that'd've": "that would have","that's": "that is", 
             "there'd": "there would", "there'd've": "there would have","there's": "there is", 
             "here's": "here is",
             "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
             "they'll've": "they will have", "they're": "they are", "they've": "they have",
             "to've": "to have", "wasn't": "was not", "we'd": "we would", 
             "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
             "we're": "we are", "we've": "we have", "weren't": "were not",
             "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
             "what's": "what is", "what've": "what have", "when's": "when is", 
             "when've": "when have", "where'd": "where did", "where's": "where is", 
             "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
             "who's": "who is", "who've": "who have", "why's": "why is", 
             "why've": "why have", "will've": "will have", "won't": "will not",
             "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
             "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
             "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
             "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
             "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [10]:
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import word_tokenize
import re
from bs4 import BeautifulSoup

tok = WordPunctTokenizer()

pat1 = r'@[\w_]+' # @-mention
pat2 = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' # URLs
pat3 = r'[:=;] [oO\-]?[D\)\]\(\]/\\OpP]' # emoticons
pat4 = r'(?:\#+[\w_]+[\w\'_\-]*[\w_]+)' # hash-tags
pat5 = r'www.[^ ]+' # additions to URLs, texts with 'www..'
combined_pat = r'|'.join((pat1, pat2, pat3,pat4,pat5))


split_pattern = re.compile(r'\b(' + '|'.join(split_dic.keys()) + r')\b')


def tweet_cleaner(demo):
    soup = BeautifulSoup(demo, 'lxml') # HTML
    souped = soup.get_text()
    try:
        clean = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
        # decoding text with 'utf-8-sig'
    except:
        clean = souped
    
    stripped = re.sub(combined_pat, '', clean)
    lower_case = stripped.lower()
    split_handled = split_pattern.sub(lambda x: split_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", split_handled)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', letters_only)
    # check if there's any spell with repeated characters such as 'soooo good', and transform it as 'soo good' or 'loove it'. Not a perfect solution but could reduct feature space by making
    words = [x for x in tok.tokenize(spell_corrected) if len(x) > 1]

    return (" ".join(words)).strip()

test_result = []
for t in demo:
    test_result.append(tweet_cleaner(t))
    # [tweet_cleaner(t) for t in demo]

test_result # cleaning texts

C:\Users\yaffy\Anaconda2\lib\encodings\utf_8_sig.py:19: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if input[:3] == codecs.BOM_UTF8:


[u'countdown final kyiv russia vs',
 u'llega los goles con el',
 u'llega los goles con el',
 u'well ended seasons campaign way started it shitty hope zidane',
 u'yall late',
 u'yall late',
 u'llega los goles con el',
 u'score goal may yall late',
 u'really hope rez brings bernab summer there player deserves',
 u'everone late ronaldo score goal real madrid may']

## Apply Text Cleaning Function to Tweets

In [11]:
df_madrid = pd.read_csv('madrid_df.csv')
replace_set = stopwords.words('english') 
df_madrid['text'] = df_madrid['text'].str.split(' ').apply(lambda x: ' '.join(k for k in x if k not in replace_set)) # replace stop-words

C:\Users\yaffy\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
df_madrid['clean_text'] = [tweet_cleaner(t) for t in df_madrid.text]

C:\Users\yaffy\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/oEJ7354EcO" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\yaffy\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/NZGnT2XvKu" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\yaffy\Anaconda2\lib\site-packages\bs4\__init__.py:282: UserWarning: "https://t.co/GOs8YOaLFe" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to

After cleaning the tweets with cleaner function, I create a new dataframe with clean text and certain columns that will be used in the further analysis

In [13]:
col_to_drop = ['text', 'extended_tweet','ideo_score']
clean_madrid = df_madrid.drop(col_to_drop, axis=1)

3,777 entries have null entries for the clean_text column. Looking at the original dataframe, those texts either with Twitter ID or URL address are identified as NaN. These are the info I decide to discard for the sentiment analysis, so I will drop these null rows, and update the data frame.

In [14]:
clean_madrid['clean_text'].replace('', np.nan, inplace=True)
print(clean_madrid['clean_text'].isna().sum())

df_madrid.iloc[clean_madrid[clean_madrid.isna().any(axis=1)].index,:].head()

3777


,created_at,name,screen_name,id,friends,followers,listed,text,extended_tweet,ideo_score,clean_text
10,Mon May 21 05:26:15 +0000 2018,Gerson Arce,19gerson,463365631.0,675,786,3,E L C O M A N D A N T E\r\r\r\n4⃣5⃣0⃣⚽ https://t.co/0NHlSdQLp9,NaN,NaN,
13,Mon May 21 06:21:00 +0000 2018,Rana Ghosal,ranaaissance,56296160.0,532,456,46,@kingpatprash @ChampionsLeague @realmadrid @LFC @sportmarketing1 @Sports_Managing @kennethcortsen @AJKarg… https://t.co/gFIrSNuPJJ,"{u'display_text_range': [133, 162], u'entities': {u'user_mentions': [{u'indices': [0, 13], u'screen_name': u'kingpatprash', u'id': 350920112, u'name': u'Prashant Pathare', u'id_str': u'350920112'}, {u'indices': [14, 30], u'screen_name': u'ChampionsLeague', u'id': 627673190, u'name': u'UEFA Champions League', u'id_str': u'627673190'}, {u'indices': [31, 42], u'screen_name': u'realmadrid', u'id': 14872237, u'name': u'Real Madrid C.F. \u26bd\ufe0f', u'id_str': u'14872237'}, {u'indices': [43, 47], u'screen_name': u'LFC', u'id': 19583545, u'name': u'Liverpool FC', u'id_str': u'19583545'}, {u'indices': [48, 64], u'screen_name': u'sportmarketing1', u'id': 27426996, u'name': u'alan seymour', u'id_str': u'27426996'}, {u'indices': [65, 81], u'screen_name': u'Sports_Managing', u'id': 873554509037015040L, u'name': u'\xd3scar Y\xe1\xf1ez', u'id_str': u'873554509037015040'}, {u'indices': [82, 97], u'screen_name': u'kennethcortsen', u'id': 24509215, u'name': u'Kenneth Cortsen', u'id_str': u'24509215'}, {u'indices': [98, 105], u'screen_name': u'AJKarg', u'id': 28756684, u'name': u'Adam Karg', u'id_str': u'28756684'}, {u'indices': [106, 121], u'screen_name': u'geoffwnjwilson', u'id': 109710458, u'name': u'Geoff Wilson', u'id_str': u'109710458'}, {u'indices': [122, 132], u'screen_name': u'Lu_Class_', u'id': 2955644493L, u'name': u'\u0141ukasz B\u0105czek', u'id_str': u'2955644493'}], u'symbols': [], u'hashtags': [], u'urls': []}, u'full_text': u'@kingpatprash @ChampionsLeague @realmadrid @LFC @sportmarketing1 @Sports_Managing @kennethcortsen @AJKarg @geoffwnjwilson @Lu_Class_ Very interesting. Optimistic?'}",NaN,
16,Mon May 21 07:06:58 +0000 2018,Sebastian Yuen ツ,S3bster,37780157.0,3136,3398,93,@boltyboy @Health2eu @bluetopaz @himsseurope @ChampionsLeague @lucienengelen @pascal_lardier @anoizet @janedwelly… https://t.co/5AWVfVgYyy,"{u'display_text_range': [196, 339], u'entities': {u'user_mentions': [{u'indices': [0, 9], u'screen_name': u'boltyboy', u'id': 1248081, u'name': u'Matthew Holt', u'id_str': u'1248081'}, {u'indices': [10, 20], u'screen_name': u'Health2eu', u'id': 221140787, u'name': u'Health 2.0 Europe', u'id_str': u'221140787'}, {u'indices': [21, 31], u'screen_name': u'bluetopaz', u'id': 1073381, u'name': u'Indu Subaiya', u'id_str': u'1073381'}, {u'indices': [32, 44], u'screen_name': u'himsseurope', u'id': 132204567, u'name': u'HIMSS Europe', u'id_str': u'132204567'}, {u'indices': [45, 61], u'screen_name': u'ChampionsLeague', u'id': 627673190, u'name': u'UEFA Champions League', u'id_str': u'627673190'}, {u'indices': [62, 76], u'screen_name': u'lucienengelen', u'id': 15123270, u'name': u'Lucien Engelen', u'id_str': u'15123270'}, {u'indices': [77, 92], u'screen_name': u'pascal_lardier', u'id': 78607265, u'name': u'Pascal Lardier', u'id_str': u'78607265'}, {u'indices': [93, 101], u'screen_name': u'anoizet', u'id': 312960789, u'name': u'Aline Noizet', u'id_str': u'312960789'}, {u'indices': [102, 113], u'screen_name': u'janedwelly', u'id': 108297471, u'name': u'Jane Dwelly', u'id_str': u'108297471'}, {u'indices': [114, 130], u'screen_name': u'Barry_HealthTec', u'id': 44844855, u'name': u'Barry Shrier', u'id_str': u'44844855'}, {u'indices': [131, 144], u'screen_name': u'HealthEugene', u'id': 16139549, u'name': u'Eugene Borukhovich', u'id_str': u'16139549'}, {u'indices': [145, 158], u'screen_name': u'chrispointon', u'id': 28578944, u'name': u'Chris Pointon', u'id_str': u'28578944'}, {u'indices': [159, 170], u'screen_name': u'Anniecoops', u'id': 20327119, u'name': u'Anne Cooper RN FQNI', u'i

In [15]:
clean_madrid.dropna(inplace=True)
clean_madrid.reset_index(drop=True,inplace=True)

In [16]:
print(clean_madrid.info())
clean_madrid.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31233 entries, 0 to 31232
Data columns (total 8 columns):
created_at     31233 non-null object
name           31233 non-null object
screen_name    31233 non-null object
id             31233 non-null float64
friends        31233 non-null int64
followers      31233 non-null int64
listed         31233 non-null int64
clean_text     31233 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 1.9+ MB
None


,created_at,name,screen_name,id,friends,followers,listed,clean_text
0,Mon May 21 01:44:58 +0000 2018,Binyamin Alsagoff,benalsagoff676,3.259756e+08,211,221,3,countdown final kyiv russia vs
1,Mon May 21 04:01:00 +0000 2018,LaLiga,LaLiga,4.233845e+08,828,3632180,5536,llega los goles con el
2,Mon May 21 04:08:40 +0000 2018,Vida Social Bogotá,SocialEnBogota,2.611749e+09,14598,34695,290,llega los goles con el
3,Mon May 21 04:38:32 +0000 2018,TD.,iamthetd,1.066035e+09,557,94,6,well ended seasons campaign way started it shitty hope zidane
4,Mon May 21 04:52:52 +0000 2018,bigtime or bigmike,Michaelgreatho2,2.440701e+09,1918,742,4,yall late


As mentioned ealier, the data was collected over four weeks since from 21 May and end at 16 June 2018 at pre-, mid- and post-event. Now I will split the texts according to the date. It can be found at the new column 'period': 

- 1: pre-event, 2018-05-21 - 2018-05-25
- 2: mid-event, 2018-05-26
- 3: post-event (week one), 2018-05-27 - 2018-06-03
- 4: post-event (week two), 2018-06-04 - 2018-06-10
- 5: post-event (week three), 2018-06-11 - 2018-06-16

Change the datetime format and keep the date part only (e.g., change 'Mon May 21 01:44:58 +0000 2018' to '2018-05-21')

In [17]:
clean_madrid['created_at'] = pd.to_datetime(clean_madrid.created_at).dt.date
clean_madrid['created_at'] = pd.to_datetime(clean_madrid.created_at, errors='coerce')

Generate a new column named 'period' and set the column values based on dates as descripted above. Save the cleaning texts with five-periods to a new dataset 'clean_madrid.csv'

In [18]:
clean_madrid['period'] = ' '
 
index1 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-05-21') & (clean_madrid['created_at'] <= '2018-05-25')].index
index2 = clean_madrid.loc[clean_madrid['created_at'] == '2018-05-26'].index
index3 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-05-27') & (clean_madrid['created_at'] <= '2018-06-03')].index
index4 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-06-04') & (clean_madrid['created_at'] <= '2018-06-10')].index
index5 = clean_madrid.loc[(clean_madrid['created_at'] >= '2018-06-11') & (clean_madrid['created_at'] <= '2018-06-16')].index
 
clean_madrid.loc[index1, 'period'] = 1
clean_madrid.loc[index2, 'period'] = 2
clean_madrid.loc[index3, 'period'] = 3
clean_madrid.loc[index4, 'period'] = 4
clean_madrid.loc[index5, 'period'] = 5
print(clean_madrid.head())

clean_madrid.to_csv('clean_madrid.csv')

,created_at,name,screen_name,id,friends,followers,listed,clean_text,period
0,2018-05-21,Binyamin Alsagoff,benalsagoff676,3.259756e+08,211,221,3,countdown final kyiv russia vs,1
1,2018-05-21,LaLiga,LaLiga,4.233845e+08,828,3632180,5536,llega los goles con el,1
2,2018-05-21,Vida Social Bogotá,SocialEnBogota,2.611749e+09,14598,34695,290,llega los goles con el,1
3,2018-05-21,TD.,iamthetd,1.066035e+09,557,94,6,well ended seasons campaign way started it shitty hope zidane,1
4,2018-05-21,bigtime or bigmike,Michaelgreatho2,2.440701e+09,1918,742,4,yall late,1
